In [1]:
import numpy as np 
import time
import os

import torch 
import torch.nn as nn
import torch.nn.functional as F 

import matplotlib.pyplot as plt
from collections import OrderedDict

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score

from utils.inference import Trainer, plot_loss
from utils.models import DNN
import utils.datasets as d

# DeepDense VAE encoder + classifier

In [2]:
X_train = np.load("data/encoded_data_train_DeepDenseVAE_mark_V_[160-256-128-15]_VDO_GNLL_400ep_lr-1e-4.npy")
y_train = np.load("data/labels_train.npy")
X_test = np.load("data/encoded_data_test_DeepDenseVAE_mark_V_[160-256-128-15]_VDO_GNLL_400ep_lr-1e-4.npy")
y_test = np.load("data/labels_test.npy")

In [3]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=666)

In [4]:
train = d.H_alphaSequences(X_train, y_train)
valid = d.H_alphaSequences(X_valid, y_valid)
test = d.H_alphaSequences(X_test, y_test)

In [5]:
train_loader = torch.utils.data.DataLoader(dataset = train, batch_size=100, shuffle=True)
valid_loader = torch.utils.data.DataLoader(dataset = valid, batch_size=100, shuffle=True)

# M.1

In [6]:
DNN1 = nn.Sequential(
            nn.Linear(in_features=15, out_features=125),
            nn.ELU(inplace=True),
            nn.Linear(in_features=125, out_features=4)
)

In [7]:
optimizer= torch.optim.Adam(DNN1.parameters(), lr=1e-3)

In [8]:
m1 = Trainer(
        model=DNN1,
        optimizer=optimizer,
        loss_function=nn.CrossEntropyLoss(),
        #scheduler=torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100], gamma=0.1),
        tensorboard=True,
        model_name="DNN_latent_space_m.I_lr-1e-3_bs-100",
        verbose=True
        )

cuda:0


In [ ]:
lh = m1(epochs=50, train_loader=train_loader, validation_loader=valid_loader)

Epoch [1/50], average_loss:0.6833, validation_loss:0.6016, val_accuracy:0.7528
Epoch [2/50], average_loss:0.5818, validation_loss:0.5700, val_accuracy:0.7978
Epoch [3/50], average_loss:0.5596, validation_loss:0.5567, val_accuracy:0.8652
Epoch [4/50], average_loss:0.5469, validation_loss:0.5471, val_accuracy:0.7865
Epoch [5/50], average_loss:0.5375, validation_loss:0.5383, val_accuracy:0.8876
Epoch [6/50], average_loss:0.5288, validation_loss:0.5318, val_accuracy:0.7978
Epoch [7/50], average_loss:0.5217, validation_loss:0.5252, val_accuracy:0.8315
Epoch [8/50], average_loss:0.5150, validation_loss:0.5220, val_accuracy:0.8202
Epoch [9/50], average_loss:0.5084, validation_loss:0.5131, val_accuracy:0.8202
Epoch [10/50], average_loss:0.5018, validation_loss:0.5071, val_accuracy:0.8652
Epoch [11/50], average_loss:0.4957, validation_loss:0.5027, val_accuracy:0.8876
Epoch [12/50], average_loss:0.4892, validation_loss:0.4989, val_accuracy:0.7978
Epoch [13/50], average_loss:0.4847, validation_lo

In [ ]:
#m1.tb.close()